In [257]:
import requests, json
from bs4 import BeautifulSoup   
import numpy as np 
import pandas as pd
import time
import datetime

In [258]:
RANDOM_SEED = 42

In [259]:
test_cars=pd.read_csv('test_cars.csv')
test_cars.head(3)

/Users/do/bin/homebrew/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11,12,16,17,19,20,32,34,35,36,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,Руль,Состояние,Таможня,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,Левый,Не требует ремонта,Растаможен,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,Левый,Не требует ремонта,Растаможен,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,Левый,Не требует ремонта,Растаможен,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [260]:
test_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              34686 non-null  object 
 1   brand                 34686 non-null  object 
 2   car_url               34686 non-null  object 
 3   color                 34686 non-null  object 
 4   complectation_dict    6418 non-null   object 
 5   description           34686 non-null  object 
 6   engineDisplacement    34686 non-null  object 
 7   enginePower           34686 non-null  object 
 8   equipment_dict        24694 non-null  object 
 9   fuelType              34683 non-null  object 
 10  image                 34686 non-null  object 
 11  mileage               34686 non-null  object 
 12  modelDate             34686 non-null  object 
 13  model_info            34686 non-null  object 
 14  model_name            34686 non-null  object 
 15  name               

In [261]:
test_cars=test_cars.drop(['car_url', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38'], axis=1)
test_cars.head(3)

,bodyType,brand,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [262]:
df_marks_and_models = pd.read_csv('all_car_brands_and_their_models_from_auto_ru.csv', index_col=0)['Models']
df_marks_and_models.head(3)

Mark
AC                     ['378_GT', 'ACE', 'ACECA', 'COBRA']
ACURA    ['CL', 'CSX', 'EL', 'ILX', 'INTEGRA', 'LEGEND'...
ADLER                        ['DIPLOMAT', 'TRUMPF_JUNIOR']
Name: Models, dtype: object

Парсинг

In [263]:
list_of_relevant_marks =list({'AC':2, 'Acura':94, 'Alfa Romeo':65, 'Alpina':10, 
                             'AMC':2, 'Ariel':1, 'Asia':1, 'Aston Martin':25, 
                             'Audi':4093, 'Austin Healey':1, 'Bajaj':3, 
                             'Bentley':104, 'BMW':7423, 'Borgward':1, 
                             'Brilliance':10, 'Buick':17, 'BYD':22, 
                             'Cadillac':321, 'Changan':100, 'Chery':593, 
                             'Chevrolet':3670, 'Chrysler':316, 'Citroen':1142, 
                             'Dacia':9, 'Daewoo':1024, 'Daihatsu':47, 
                             'Daimler':5, 'Datsun':212, 'Delage':1, 'Derways':2, 
                             'DKW':1, 'Dodge':413, 'DongFeng':25, 'Doninvest':2, 
                             'DS':3, 'DW Hower':8, 'Eagle':1, 'Excalibur':1, 
                             'FAW':66, 'Ferrari':29, 'Fiat':206, 'Ford':4812, 
                             'Foton':2, 'GAC':14, 'Geely':302, 'Genesis':54, 
                             'GMC':26, 'Great Wall':420, 'Hafei':6, 'Haima':12, 
                             'Haval':110, 'Hawtai':4, 'Honda':1179, 
                             'HuangHai':1, 'Hummer':70, 'Hyundai':5712, 
                             'Infiniti':1041, 'Innocenti':1, 'Iran Khodro':21, 
                             'Isuzu':43, 'JAC':8, 'Jaguar':308, 'Jeep':466, 'JMC':1, 
                             'Kia':5595, 'Lamborghini':14, 
                             'Lancia':8, 'Land Rover':1616, 'Lexus':985, 
                             'Lifan':295, 'Lincoln':62, 'Lotus':1, 'Luxgen':9, 
                             'Mahindra':1, 'Maserati':54, 'Maybach':7, 
                             'Mazda':2311, 'McLaren':5, 'MERCEDES':8005, 
                             'Mercury':16, 'Metrocab':2, 'MG':10, 'MINI':313, 
                             'Mitsubishi':3935, 'Mitsuoka':3, 'Nissan':5776, 
                             'Oldsmobile':10, 'Opel':3131, 'Packard':2, 
                             'Peugeot':1636, 'Plymouth':13, 'Pontiac':38, 
                             'Porsche':714, 'Proton':3, 'PUCH':1, 'Ravon':112, 
                             'Renault':2771, 'Rolls-Royce':55, 'Rover':69, 
                             'Saab':147, 'Saturn':9, 'Scion':9, 'SEAT':77, 
                             'Shanghai Maple':1, 'Skoda':2810, 'Smart':89, 
                             'SsangYong':718, 'Subaru':934, 'Suzuki':878, 
                             'Tatra':5, 'Tesla':43, 'Tianma':2, 'Toyota':4648, 
                             'Triumph':1, 'Vauxhall':1, 'Volkswagen':7109, 
                             'Volvo':2069, 'Vortex':91, 'Wartburg':1, 'Willys':2, 
                             'Zotye':15, 'ZX':4, 'ГАЗ':935, 'Гоночный болид':2, 
                             'ЗАЗ':231, 'ЗИЛ':2, 'ИЖ':129, 'ЛуАЗ':45, 
                             'Москвич':198, 'СМЗ':4, 'ТагАЗ':68, 'УАЗ':1107}.keys())

In [264]:
HEADERS = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Connection': 'keep-alive',
    
    'content-type': 'application/json',
    'Cookie': '_csrf_token=d8029e40b7e75f491e05b9191bce2b0854cb8560d32b9010; autoru_sid=a%3Ag5fcbae7b2rh31h6p5i54qkso13bmhrs.d64fdb1048094ade6653bb8d77bfc82f%7C1607183995440.604800.lS7lenbdJu7TVJgQ9a2fBw.M4uhYka1yjkPXI7TXYAIHoc1bAMl16bHMa5Rel1JyiM; autoruuid=g5fcbae7b2rh31h6p5i54qkso13bmhrs.d64fdb1048094ade6653bb8d77bfc82f; autoru_gdpr=1; suid=bc138c1d5a6554d5f028f6fec11a91da.a9ca18211fb27ebefea2fe04a11546b6; from=direct; yuidcs=1; crookie=aAs/ROiBsnuk/909OZNBhxIcXPqhyXpNRRJ0N6Sk1kI/Eaknz7W1f3rW2MUp05I5lp/zbVBPKBkNgTKZqk8ULF8Q0Vg=; cmtchd=MTYwNzI1MDk3ODU4MQ==; popup_new_user=new; proven_owner_popup=1; bltsr=1; yuidlt=1; yandexuid=1370723801540629363; X-Vertis-DC=vla; listing_view_session={}; listing_view=%7B%22output_type%22%3A%22models_list%22%2C%22version%22%3A1%7D; new-controls-notify-hidden=true; from_lifetime=1607289350518',
    'Host': 'auto.ru',
    'origin': 'https://auto.ru/cars/all/',
    'Referer': 'https://auto.ru/cars/all/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
    'x-client-app-version': '202012.04.170215',
    'x-client-date': '1607289359144',
    'x-csrf-token': 'd8029e40b7e75f491e05b9191bce2b0854cb8560d32b9010',
    'x-page-request-id': '52d78c631a6cdbf362cebe6be367d327',
    'x-requested-with': 'fetch'
}

In [265]:
url='https://auto.ru/-/ajax/desktop/listing/'

In [266]:
auto_ru = {}
# сделаем тестовый вызов запроса для BMW модели M6 и выведем его на экран, чтобы по нему можно было сделать функцию для создания 23 необходимых столбцов


# но сначала установим главные параметры запроса
mark = 'SKODA'  # марка
model = 'OCTAVIA'  # модель
geo_radius = 200  # радиус вокруг москвы = 200 км
geo_id = [213]  # Москва
top_days = "1"  # кол-во дней от текущей даты назад за которые запрашиваем объявления 

req_body = {
                'catalog_filter' : [{"mark": mark, "model": model}],
                'section': "all",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "top_days": "900",
                "geo_radius":geo_radius,
                "geo_id":geo_id,
                'page': 1
            }

response = requests.post(url, json=req_body, headers = HEADERS)
data = response.json()
for elem in data['offers']:
    ad = {
                        'bodyType': elem['vehicle_info']['configuration']['body_type'],
                        'brand': mark, 
                        #'car_url',
                        'color': elem['vehicle_info']['complectation']['vendor_colors']0]['name_ru'],
                        'complectation_dict': '',
                        'description': '', 
                        'engineDisplacement': '',
                        'enginePower': '', 
                        'equipment_dict': '',
                        'fuelType': '', 
                        'image': '', 
                        'mileage': '', 
                        'modelDate': '',
                        'model_info': '',
                        'model_name': '',
                        'name': '', 
                        'numberOfDoors': '',
                        'parsing_unixtime': '',
                        'priceCurrency': '',
                        'productionDate': '', 
                        'sell_id': '', 
                        'super_gen': '',
                        'vehicleConfiguration': '',
                        'vehicleTransmission': '', 
                        'vendor': '',
                        'Владельцы': '',
                        'Владение': '', 
                        'ПТС': '',
                        'Привод': '', 
                        'Руль': '', 
                        'Состояние': '',
                        'Таможня': ''}
    print(ad)
    auto_ru[0] = ad
    break
    
#df_auto_ru = pd.DataFrame(auto_ru).T
#df_auto_ru.to_csv('all_auto_ru_07_12_2020-5.csv', index=False)

{'bodyType': 'LIFTBACK', 'brand': 'SKODA', 'color': 'Серый Platinum', 'complectation_dict': '', 'description': '', 'engineDisplacement': '', 'enginePower': '', 'equipment_dict': '', 'fuelType': '', 'image': '', 'mileage': '', 'modelDate': '', 'model_info': '', 'model_name': '', 'name': '', 'numberOfDoors': '', 'parsing_unixtime': '', 'priceCurrency': '', 'productionDate': '', 'sell_id': '', 'super_gen': '', 'vehicleConfiguration': '', 'vehicleTransmission': '', 'vendor': '', 'Владельцы': '', 'Владение': '', 'ПТС': '', 'Привод': '', 'Руль': '', 'Состояние': '', 'Таможня': ''}


In [287]:
iter_of_ads, iter_of_marks, old_iter_of_ads, sum_ads, sum_ads_from_dict = 0,0,0,0,0
space = ' '
start_time_global = time.ctime()
for mark in list_of_relevant_marks:
    start_time = time.ctime()
    list_of_models = json.loads(df_marks_and_models.loc[mark.upper().replace(" ", "_").replace("-", "_")].replace("'", '"'))
    for model in list_of_models:
        for num_page in range(1, 2):
            param = {
                'catalog_filter' : [{"mark": mark, "model": model}],
                'section': "all",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "top_days":"900",
                "geo_radius":200,
                "geo_id":[213],
                'page': num_page
            }
            response = requests.post(url, json=param, headers = HEADERS)
            data = response.json()
            if data['offers'] != []:
                for elem in data['offers']:
                    # print(elem)
                    def from_elem_to_dict(d_elem, d_dict):
                        try: d_dict['bodyType'] = d_elem['vehicle_info']['configuration']['human_name']
                        except: d_dict['bodyType'] = None

                        try: d_dict['brand'] = d_elem['vehicle_info']['mark_info']['code']
                        except: d_dict['brand'] = None

                        try: d_dict['color'] = d_elem['color_hex']
                        except: d_dict['color'] = None

                        try: d_dict['fuelType'] = d_elem['lk_summary'].split()[-1]
                        except: d_dict['fuelType'] = None

                        try: d_dict['modelDate'] = d_elem['vehicle_info']['super_gen']['year_from']
                        except: d_dict['modelDate'] = None

                        try: d_dict['name'] =  d_elem['vehicle_info']['tech_param']['human_name']
                        except: d_dict['name'] = None

                        try: d_dict['numberOfDoors'] = d_elem['vehicle_info']['configuration']['doors_count']
                        except: d_dict['numberOfDoors'] = None

                        try: d_dict['productionDate'] = d_elem['documents']['year']
                        except: d_dict['productionDate'] = None

                        try: d_dict['vehicleTransmission'] = d_elem['vehicle_info']['tech_param']['transmission']
                        except: d_dict['vehicleTransmission'] = None

                        try: d_dict['engineDisplacement'] = d_elem['vehicle_info']['tech_param']['human_name'].split()[0]
                        except: d_dict['engineDisplacement'] = None

                        try: d_dict['vehicleConfiguration'] = elem['vehicle_info']['configuration']['body_type']+' '+d_dict['vehicleTransmission']+' '+d_dict['engineDisplacement']
                        except: d_dict['vehicleConfiguration'] = None

                        try: d_dict['enginePower'] = d_elem['vehicle_info']['tech_param']['power']
                        except: d_dict['enginePower'] = None

                        try: d_dict['description'] = d_elem['description']
                        except: d_dict['description'] = None

                        try: d_dict['mileage'] = d_elem['state']['mileage']
                        except: d_dict['mileage'] = None

                        try: d_dict['Комплектация'] = d_elem['vehicle_info']['complectation']
                        except: d_dict['Комплектация'] = None

                        try: d_dict['Привод'] = d_elem['lk_summary'].split(', ')[-2]
                        except: d_dict['Привод'] = None

                        try: d_dict['Руль'] = d_elem['vehicle_info']['steering_wheel']
                        except: d_dict['Руль'] = None

                        # 'Состояние' не найдено
                        d_dict['Состояние'] = None

                        try: d_dict['Владельцы'] = d_elem['documents']['owners_number']
                        except: d_dict['Владельцы'] = None

                        try: d_dict['ПТС'] = d_elem['documents']['pts']
                        except: d_dict['ПТС'] = None

                        try: d_dict['Таможня'] = d_elem['documents']['custom_cleared']
                        except: d_dict['Таможня'] = None

                        try: d_dict['Владение'] =  d_elem['documents']['purchase_date']
                        except: d_dict['Владение'] = None


                        try: d_dict['price'] = d_elem['price_info']['RUR']
                        except: d_dict['price'] = None

                        # дата размещения объявления решил добавить 
                        try: d_dict['start_date'] =  d_elem['additional_info']['hot_info']['start_time']
                        except: d_dict['start_date'] = None

                        # статус объявления 
                        try: d_dict['hidden'] =  d_elemelem['additional_info']['hidden']
                        except: d_dict['hidden'] = None

                        # модель 
                        d_dict['model'] =  model

                        return d_dict
                    '''
                    ad = {
                        'bodyType': elem['vehicle_info']['configuration']['body_type'],
                        'brand': mark, 
                        #'car_url',
                        'color': color(),
                        'complectation_dict': '',
                        'description': '', 
                        'engineDisplacement': '',
                        'enginePower': '', 
                        'equipment_dict': '',
                        'fuelType': '', 
                        'image': '', 
                        'mileage': '', 
                        'modelDate': '',
                        'model_info': '',
                        'model_name': '',
                        'name': '', 
                        'numberOfDoors': '',
                        'parsing_unixtime': '',
                        'priceCurrency': '',
                        'productionDate': '', 
                        'sell_id': '', 
                        'super_gen': '',
                        'vehicleConfiguration': '',
                        'vehicleTransmission': '', 
                        'vendor': '',
                        'Владельцы': '',
                        'Владение': '', 
                        'ПТС': '',
                        'Привод': '', 
                        'Руль': '', 
                        'Состояние': '',
                        'Таможня': ''}
                    '''
                    auto_ru[iter_of_ads] = from_elem_to_dict(elem, {})
                    print('.', end='')
                    iter_of_ads += 1
            else:
                break
print (f'Всего спарсили := {iter_of_ads} строк, на это ушло: {int((time.ctime() - start_time_global)/60)} мин., ')            

................................................................................................................................................................................

KeyboardInterrupt: 

In [288]:
auto_ru

{0: {'bodyType': 'Родстер',
  'brand': 'AC',
  'color': 'FAFBFB',
  'fuelType': 'бензин',
  'modelDate': 1999,
  'name': '4.9 MT (340 л.с.)',
  'numberOfDoors': 2,
  'productionDate': 1999,
  'vehicleTransmission': 'MECHANICAL',
  'engineDisplacement': '4.9',
  'vehicleConfiguration': 'ROADSTER MECHANICAL 4.9',
  'enginePower': 340,
  'description': 'Shelby AC Cobra\nОдна из самых быстрых Кобр\nСпецификация:\nДвигатель-351 Windsor  (418 куб.дюймов) профессионально изготовлен NASCAR производителями. Более $ 35 000 инвестировано только на трансмиссию и двигатель.\nТрансмиссия- 2-скоростная\nПодвеска- Ford Mestang 11 дюймов 4 дисковые тормоза\nКузов. опара- стальные трубы спец. фирмы\nСалон-плюш два тона бежевого, специального изготовления\nПокраска-белая с золотыми полосами Craven Auto Body\nРезина- Goodrich Drag Radials P235-60R15 Front/P325-50R15 Real\nДиски-полированное золото by Team III Wheels/Powdertech Plus\nБоковые трубы- нержавеющая сталь by Baxter Custom Engineering\nЗвуковая с

In [289]:
df_auto_ru = pd.DataFrame(auto_ru).T
df_auto_ru.to_csv('all_auto_ru_07_12_2020-5.csv', index=False)
df_auto_ru.head
#print(f'кол-во строк датасета:= {len(df_auto_ru)}')

<bound method NDFrame.head of             bodyType brand   color fuelType modelDate                    name  \
0            Родстер    AC  FAFBFB   бензин      1999       4.9 MT (340 л.с.)   
1            Родстер    AC  EE1D19   бензин      1990       4.9 MT (326 л.с.)   
2    Универсал 5 дв.   AMC  EE1D19   бензин      1979    2.5 MT (85 л.с.) 4WD   
3        Седан 2 дв.   BMW  FF8649   бензин      1966       2.0 MT (101 л.с.)   
4      Хэтчбек 5 дв.   BMW  97948F   бензин      2011  118i 1.6 AT (170 л.с.)   
..               ...   ...     ...      ...       ...                     ...   
171          Родстер   BMW  FFD600   бензин      2009   23i 2.5 AT (204 л.с.)   
172             Купе   BMW  0000CC   бензин      2005       3.0 AT (265 л.с.)   
173          Родстер   BMW  040001   бензин      2005       3.0 AT (265 л.с.)   
174          Родстер   BMW  EE1D19   бензин      2002       3.0 MT (231 л.с.)   
175          Родстер   BMW  FF8649   бензин      2013   20i 2.0 AT (184 л.с.)  

In [290]:
print(len(test_cars))
frames = [test_cars, df_auto_ru]
result = pd.concat(frames)
print(len(result))

34686
34862


#POST /-/ajax/desktop/listing/ HTTP/1.1
Host: auto.ru
Connection: keep-alive
Content-Length: 176
x-requested-with: fetch
x-client-date: 1607289359144
x-csrf-token: d8029e40b7e75f491e05b9191bce2b0854cb8560d32b9010
x-page-request-id: 52d78c631a6cdbf362cebe6be367d327
content-type: application/json
User-Agent: Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Mobile Safari/537.36
x-client-app-version: 202012.04.170215
Accept: */*
Origin: https://auto.ru
Sec-Fetch-Site: same-origin
Sec-Fetch-Mode: same-origin
Sec-Fetch-Dest: empty
Referer: https://auto.ru/moskva/cars/skoda/octavia-octavia/all/
Accept-Encoding: gzip, deflate, br
Accept-Language: de-DE,de;q=0.9,en-US;q=0.8,en;q=0.7
Cookie: _csrf_token=d8029e40b7e75f491e05b9191bce2b0854cb8560d32b9010; autoru_sid=a%3Ag5fcbae7b2rh31h6p5i54qkso13bmhrs.d64fdb1048094ade6653bb8d77bfc82f%7C1607183995440.604800.lS7lenbdJu7TVJgQ9a2fBw.M4uhYka1yjkPXI7TXYAIHoc1bAMl16bHMa5Rel1JyiM; autoruuid=g5fcbae7b2rh31h6p5i54qkso13bmhrs.d64fdb1048094ade6653bb8d77bfc82f; autoru_gdpr=1; suid=bc138c1d5a6554d5f028f6fec11a91da.a9ca18211fb27ebefea2fe04a11546b6; from=direct; yuidcs=1; crookie=aAs/ROiBsnuk/909OZNBhxIcXPqhyXpNRRJ0N6Sk1kI/Eaknz7W1f3rW2MUp05I5lp/zbVBPKBkNgTKZqk8ULF8Q0Vg=; cmtchd=MTYwNzI1MDk3ODU4MQ==; popup_new_user=new; proven_owner_popup=1; bltsr=1; yuidlt=1; yandexuid=1370723801540629363; X-Vertis-DC=vla; listing_view_session={}; listing_view=%7B%22output_type%22%3A%22models_list%22%2C%22version%22%3A1%7D; new-controls-notify-hidden=true; from_lifetime=1607289350518

In [ ]:
# {"catalog_filter":[{"mark":"SKODA","model":"OCTAVIA","nameplate_name":"octavia"}],"section":"all","category":"cars","output_type":"models_list","geo_radius":200,"geo_id":[213]}

https://auto.ru/-/ajax/desktop/listing/

In [ ]:
{
  "catalog_filter": [
    {
      "mark": "SKODA",
      "model": "OCTAVIA",
      "nameplate_name": "octavia"
    }
  ],
  "section": "all",
  "category": "cars",
  "output_type": "models_list",
  "geo_radius": 200,
  "geo_id": [
    213
  ]
}